This is the large casestudy for the damage dataset. Data that we are reading is 34Gb and device that I have is 32 GB so technically I cannot play around with this data

In [1]:
from dask.distributed import Client, progress,wait
import tables
import numpy as np


In [2]:
import dask.array as da

In [3]:
path="/home/berkay/Projects/pymks/data/ms_copy_data.hdf5"


In [4]:
tps_hdf5_file = tables.open_file(path, mode='r')


In [5]:
def calc_2ps(phase):
    global ndim
    from pymks.stats import correlate
    from pymks import PrimitiveBasis
    import numpy as np
    prim_basis=PrimitiveBasis(n_states=2)
    interface=phase[ndim**3:]
    phase=phase[:ndim**3]
    phase_mks=phase.reshape((1,ndim,ndim,ndim))
    interface_mks=interface.reshape((1,ndim-1,ndim-1,ndim-1))
    phase_auto=correlate(phase_mks,prim_basis,periodic_axes=(0,1,2))
    auto_11=phase_auto[0,:,:,:,0]
    intcorr=correlate(interface_mks,prim_basis,periodic_axes=(0,1,2))
    autoint=intcorr[0,:,:,:,0]
    autoint=1-autoint
    tot_2ps=np.column_stack([np.reshape(auto_11,(1,ndim**3)),\
                             np.reshape(autoint,(1,(ndim-1)**3))])
    return tot_2ps

In [6]:
nsample=1000
ndim=51
chunksize=1300
ndim_data=da.from_array(np.repeat(ndim,nsample).T,chunks=(chunksize,))

In [7]:
ms_data=tps_hdf5_file.root.ms_data

In [8]:
# %timeit MicrostructureData=Full_Data[:,0:51**3+1]

In [1]:
# data=da.from_array(Full_Data,chunks=(1300,257651))

In [12]:
for i in range(1):
    client=Client()
    ms_chunk=np.array([ms_data[i*chunksize:(i+1)*chunksize]])
    ms_chunk=ms_chunk.reshape((chunksize,ms_data.shape[1]))
    # ms_chunk=da.from_array(ms_chunk)
    futures=client.map(calc_2ps,ms_chunk)
    wait(futures)
    tot_2ps=client.gather(futures)
    tot_2ps=np.stack(tot_2ps)
    tot_2ps=tot_2ps.reshape((chunksize,ms_data.shape[1]))
    client.close()
#         tps_data_storage.append(tot_2ps)
#         tps_data_storage.flush()
    # tot_2ps=da.map_blocks(calc_2ps,ms_chunk).compute()
    if(i==0):
        print(tot_2ps)
    # tps_data[i*chunksize:(i+1)*chunksize,ms_data.shape[1],:]=da.asarray(tot_2ps.reshape((chunksize,ms_data.shape[1])))
    # tps_data[i*chunksize:(i+1)*chunksize,ms_data.shape[1],:]=da.from_array(tot_2ps.reshape((chunksize,ms_data.shape[1])))
    print("%d Chunks completed"%(i+1))

/nix/store/02z0vx25y0x2arfvcp89pvy52a3q77wf-python3.7-distributed-2.9.1/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


[[0.27861833 0.2781434  0.27723123 ... 0.580456   0.580808   0.580992  ]
 [0.27745739 0.27855048 0.27940988 ... 0.577224   0.577376   0.5778    ]
 [0.27674876 0.27571598 0.27582906 ... 0.579144   0.580592   0.5798    ]
 ...
 [0.26429503 0.26529012 0.26804924 ... 0.584312   0.584152   0.584888  ]
 [0.27831679 0.2790028  0.27947019 ... 0.57748    0.576512   0.575848  ]
 [0.27186376 0.27049928 0.26819247 ... 0.586736   0.587096   0.5884    ]]
1 Chunks completed


In [13]:
tot_2ps.shape

(1300, 257651)

In [ ]:
# from pymks.fmks.bases.primitive import PrimitiveTransformer
# from pymks.fmks.correlations import TwoPointCorrelation,FlattenTransformer
# # from sklearn.decomposition import PCA
# from sklearn.pipeline import Pipeline


In [ ]:
# bl = Pipeline(steps=[
#     ("discritize",PrimitiveTransformer(n_state=2, min_=0.0, max_=1.0)),
#     ("Correlations",TwoPointCorrelation(periodic_boundary=True, cutoff=25,correlations=[(1,1),(0,1)])),
#     ('flatten', FlattenTransformer()),  
#     ('reducer',PCA())])

